In [1]:
from collections import namedtuple
from enum import Enum
from typing import Dict, List

with open("input.txt") as f:
    lines = [l.strip() for l in f.readlines()]

In [2]:
Point = namedtuple("Point", "x y")

class Element(Enum):
    ROCK = 1
    SAND = 2

def line_to_points(l: str) -> List[Point]:
    """Parses a line in the format '498,4 -> 498,6 -> 496,6' into list of points"""
    parts = l.split("->")
    points = []
    for part in parts:
        x, y = part.split(",")
        points.append(Point(int(x), int(y)))
    return points

def all_points_inclusive(start, end: Point) -> List[Point]:
    """Take start and end point, return all points between them with start and end inclusive"""
    points = []
    if start.x == end.x:
        min_y, max_y = min(start.y, end.y), max(start.y, end.y)
        return [
            Point(start.x, y) for y in range(min_y, max_y+1)
        ]

    min_x, max_x = min(start.x, end.x), max(start.x, end.x)
    return [
        Point(x, start.y) for x in range(min_x, max_x+1)
    ]
 

def init_grid(lines: List[str]) -> Dict[Point, Element]:
    grid = {}
    for l in lines:
        parsed = line_to_points(l)
        for i in range(len(parsed)-1):
            points = all_points_inclusive(parsed[i], parsed[i+1])
            
            for point in points:
                grid[point] = Element.ROCK
    
    return grid

def count_sand(grid: Dict[Point, Element]) -> int:
    return sum(el == Element.SAND for el in grid.values())

In [3]:
def simulate_part1(grid: Dict[Point, Element]) -> Dict[Point, Element]:
    """Run simulation until water overflows"""
    # find lowest Y point, and once we have sand that
    # passes the lowest Y point, then we are done
    lowest_y = float("-inf")
    for p in grid.keys():
        lowest_y = max(lowest_y, p.y)

    while True:
        # start drop at (500, 0) and apply rules until it comes to rest or passes lowest_y
        p = Point(500, 0)
        while True:
            if p.y > lowest_y:
                return grid

            if Point(p.x, p.y+1) not in grid:
                p = Point(p.x, p.y+1)
            elif Point(p.x-1, p.y+1) not in grid:
                p = Point(p.x-1, p.y+1)
            elif Point(p.x+1, p.y+1) not in grid:
                p = Point(p.x+1, p.y+1)
            else:
                grid[p] = Element.SAND
                break


f"part 1: {count_sand(simulate_part1(init_grid(lines)))}"

'part 1: 892'

In [4]:
def simulate_part2(grid: Dict[Point, Element]) -> Dict[Point, Element]:
    """Run simulation until water reaches (500, 0)"""
    start = Point(500, 0)
    # find lowest Y point, and floor is 2 lower than lowest y
    lowest_y = float("-inf")
    for p in grid.keys():
        lowest_y = max(lowest_y, p.y)

    floor_y = lowest_y + 2

    while start not in grid:
        p = start
        while True:
            if p.y + 1 == floor_y:
                grid[p] = Element.SAND
                break

            if Point(p.x, p.y+1) not in grid:
                p = Point(p.x, p.y+1)
            elif Point(p.x-1, p.y+1) not in grid:
                p = Point(p.x-1, p.y+1)
            elif Point(p.x+1, p.y+1) not in grid:
                p = Point(p.x+1, p.y+1)
            else:
                grid[p] = Element.SAND
                break

    return grid


f"part 2: {count_sand(simulate_part2(init_grid(lines)))}"

'part 2: 27155'